In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"
3,290620657799159809,Anne Hathaway has got me living. #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': '_NicoleEdwards', 'id': 144430..."
4,290620657778188288,Jennifer Lopez's lace dress? Thoughts? #Golden...,2013-01-14 00:45:38,"{'screen_name': 'lolaogunnaike', 'id': 134953223}"
5,290620657719455745,Podrán criticar a #Adele de su moda y su maniq...,2013-01-14 00:45:38,"{'screen_name': 'SabyOnford', 'id': 404554221}"
6,290620657715273728,"US, Weakly #GoldenGlobes",2013-01-14 00:45:38,"{'screen_name': 'alejandraoleary', 'id': 45748..."
7,290620657715253248,"RT @BillMc7: ""Wait. What's that smell?!"" (ever...",2013-01-14 00:45:38,"{'screen_name': 'xtinavickers', 'id': 92139052}"
8,290620657706872832,Hugh Jackman is so awesome!!! #goldenglobes,2013-01-14 00:45:38,"{'screen_name': 'Boaz_173', 'id': 31504713}"
9,290620657560084480,It was awkward. RT @hollywoodhwife: They cut t...,2013-01-14 00:45:38,"{'screen_name': 'KateSpencer1', 'id': 21571382}"


In [3]:
len(df)

174643

In [4]:
data = df['text']
data[:10]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
5    Podrán criticar a #Adele de su moda y su maniq...
6                             US, Weakly #GoldenGlobes
7    RT @BillMc7: "Wait. What's that smell?!" (ever...
8          Hugh Jackman is so awesome!!! #goldenglobes
9    It was awkward. RT @hollywoodhwife: They cut t...
Name: text, dtype: object

In [5]:
data[1]

"What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes"

In [6]:
import re

In [147]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [8]:
# test
for line in data[:10]:
    print(line)
    print(cleanse(line))
    print(line.split())

JLo's dress! #eredcarpet #GoldenGlobes
jlo's dress  #eredcarpet #goldenglobes
["JLo's", 'dress!', '#eredcarpet', '#GoldenGlobes']

What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes
what's making sofia vergara's boobs stay like that  magic  witchcraft  #goldenglobes
["What's", 'making', 'Sofia', "Vergara's", 'boobs', 'stay', 'like', 'that?', 'Magic?', 'Witchcraft?', '#GoldenGlobes']

RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO
rt @fabsugar  kerry washington is everything  dying over her miu miu gown  @goldenglobes #goldenglobes http   t co 3feh7mvo
['RT', '@FabSugar:', 'Kerry', 'Washington', 'is', 'EVERYTHING.', 'Dying', 'over', 'her', 'Miu', 'Miu', 'gown!', '@goldenglobes', '#goldenglobes', 'http://t.co/3feH7MvO']

Anne Hathaway has got me living. #GoldenGlobes
anne hathaway has got me living  #goldenglobes
['Anne', 'Hathaway', 'has', 'got', 'me', 'living.', '#GoldenGlobes

In [98]:
import spacy
# python -m spacy download en
nlp = spacy.load('en')

In [225]:
def identify_entities(text, verbose=False):
    ents = list(nlp(text).ents)
    if verbose:
        tags = {}
        for ent in ents:    
            entity = ' '.join(t.orth_ for t in ent).strip()
            if entity not in tags:
                tags[entity]=[ent.label_]
        return tags
    else:
        return [' '.join(t.orth_ for t in ent).strip() for ent in ents]

In [241]:
# test
import pprint
text = "“We’re going to keep things loose,” said Amy Poehler of her and co-host Tina Fey’s plan for the evening. #GoldenGlobes http://t.co/o40g5LGq“"
pprint.pprint(identify_entities(text, True))
print(identify_entities(text, False))

{'Amy Poehler': ['PERSON'],
 'GoldenGlobes': ['MONEY'],
 'Tina Fey ’s': ['PERSON'],
 'the evening': ['TIME']}
['Amy Poehler', 'Tina Fey ’s', 'the evening', 'GoldenGlobes']


In [236]:
def find_host(verbose=False):
    pattern = re.compile(r'host')
    entity_dict = {}
    num = 0
    
    for line in data:
        match = re.findall(pattern, cleanse(line).lower())
        if match:
            entities = identify_entities(cleanse(line), False)

            if verbose:
                # print the first 5 occurrences
                if num < 5:
                    print(match)
                    print(line)
                    print(entities)
                    print()

            for entity in entities:
                if len(entity) > 1:
                    if entity not in entity_dict:
                        entity_dict[entity] = 1
                    else:
                        entity_dict[entity] += 1

            num += 1
    print('num of matches:', num)
    print('top inferences:', list(entity_dict.keys())[:3], '\n')

In [237]:
%%time
find_host(verbose=True)

['host']
Looking forward to watching Tina Fey and Amy Poehler host the #GoldenGlobes
['Tina Fey', 'Amy Poehler']

['host']
RT @goldenglobes: It's our hosts Tina Fey and Amy Poehler! #goldenglobes #redcarpet http://t.co/8lqC3ocQ
['RT @goldenglobes', 'Tina Fey', 'Amy Poehler', '#']

['host']
RT @washingtonpost: Tonight's dual hosting duties represent the culmination of a decade of Amy and Tina partnerships. http://t.co/scSThrn5 #GoldenGlobes
['RT @washingtonpost', 'Tonight', 'a decade', 'Amy', 'Tina', '']

['host']
RT @MovieMayor: My green suede tuxedo pinching a bit here at the Velvet Rope Awards honoring best in crowd control. Topo Gigio + I hosting #GoldenGlobes
['RT @MovieMayor', 'the Velvet Rope Awards', '', 'Topo Gigio', 'GoldenGlobes']

['host']
RT @accesshollywood: Tina Fey &amp; Amy Poehler Talk #GoldenGlobes Hosting, Drinking Game  http://t.co/1mIvq362
['RT @accesshollywood', 'Amy Poehler Talk', 'GoldenGlobes Hosting', 'Drinking Game', '']

num of matches: 3109
top inferences: 

In [ ]:
# pattern = re.compile(r'jennifer lopez')